# Video Q/A with Videoprism and Parakeet-v3

<a target="_blank" href="https://colab.research.google.com/github/everettVT/daft-video-embeddings/blob/main/workload/video_embeddings_videoprism.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

Videoprism is a general-purpose video encoder designed to tackle a wide spectrum of video understanding tasks, including classification localization, retrieval, captioning, and question answering.

Parakeet is a 600-million-parameter multilingual automatic speech recognition (ASR) model designed for high-throughput speech-to-text transcription.

In this notebook, we will explore how to leverage these foundational models to generate video and text embeddings from youtube or any video file.

Video processing requires us to extract both image and audio frames, which can then use to generate embeddings. In this use case we will be transcribing the audio to text segments so we can perform RAG Q/A against both the visual and spoken content.


In [1]:
# @title Prepare environment

import os
import sys

# Fetch VideoPrism repository if Python does not know about it and install
# dependencies needed for this notebook.
if not os.path.exists("videoprism_repo"):
  !git clone --quiet --branch=main --depth=1 \
     https://github.com/everettVT/videoprism.git videoprism_repo
  os.chdir('./videoprism_repo')
  !pip install .
  os.chdir('..')

# Append VideoPrism code to Python import path.
if "videoprism_repo" not in sys.path:
  sys.path.append("videoprism_repo")

Processing /content/videoprism_repo
  Preparing metadata (setup.py) ... done
  Created wheel for videoprism: filename=videoprism-1.0.0-py3-none-any.whl size=40354 sha256=9e5f435cc343b496574d6d3be418a143269d714254c88a0e2fb8d0e1cdf14e85
  Stored in directory: /tmp/pip-ephem-wheel-cache-xmbly3et/wheels/e3/73/3c/3dc3551ff92b46a1e55f9a893f2d5b8fdc55d670bd73d3b605
Successfully built videoprism


In [2]:
!pip install -q "daft>=0.6.1" av yt-dlp "jax[cuda12]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.1/177.1 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 MB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 61.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 100.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 MB 57.3 MB/s eta 0:00:00


In [3]:
!apt-get update -qq
!apt-get install -y -qq ffmpeg
!ffprobe -version

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
ffprobe version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2007-2021 the FFmpeg developers
built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine 

In [2]:
import daft
from daft import col, DataType as dt
import numpy as np
import jax
import jax.numpy as jnp
from jax.extend import backend
import tensorflow as tf
from videoprism import models as vp
print(jax.devices())    # should list a CUDA device

[CudaDevice(id=0)]


### Define Parameters

Tensor Dimensions:
- B: batch size (number of videos in a batch).
- T: number of frames per video clip (typically 16).
- N: tokens per frame (for 288×288 with 18×18 patches → 16×16 = 256).
- D: embedding dimension (Base: 768; Large: 1024).

VideoPrism supports video+text inputs and returns:
- video_embeddings: [B, D] (global video embeddings).
- text_embeddings: [B, D] (global text embeddings).
- Optional: frame_embeddings [B, T, D]; tokens [B, T×N, D]

In [8]:
import os


B, T, H, W, C = 2, 16, 288, 288, 3
ROW_LIMIT = 2048
VIDEO_DIR = f"{os.getcwd()}/videos" # Overwrite with desired

### Upload/Download videos o

```bash
!yt-dlp -P /content/videos -f 'bv*[ext=mp4]+ba[ext=m4a]/b[ext=mp4]' \
  https://www.youtube.com/watch?v=WAsmZJ2kff0 \
  https://www.youtube.com/watch?v=BLcKDQRTFKY \
  https://www.youtube.com/watch?v=Qnw6059ddgE \
  https://www.youtube.com/watch?v=eYXDSuNpKTk \
  https://www.youtube.com/watch?v=3JWrg1DitaA
```


### Discover video files

In [13]:
from daft.functions import file

df_files = daft.from_glob_path(VIDEO_DIR).with_column("file", file(col("path")))

df_files.show(5)

pathUtf8,sizeInt64,num_rowsInt64,fileFile
file:///content/videos/Running.mp4,132936424,None,"File(Reference(""file:///content/videos/Running.mp4"", None))"


Retrieve Metadata from file headers

In [18]:
import subprocess, json
from fractions import Fraction

out = subprocess.check_output([
    "ffprobe","-v","error",
    "-select_streams","v:0",
    "-show_streams","-show_format",
    "-print_format","json", str("file:///content/videos/Running.mp4")
])
info = json.loads(out)

# Cast ffprobe output to our dtype
w = info["streams"][0]["width"]
h = info["streams"][0]["height"]
fps = float(Fraction(info["streams"][0]["r_frame_rate"]))
nb_frames = info["streams"][0]["nb_frames"]
time_base = float(Fraction(info["streams"][0]["time_base"]))

print( {
        "width": w,
        "height": h,
        "fps": fps,
        "frame_count": nb_frames,
        "time_base": time_base,
        "json": json.dumps(info),
    })

{'width': 1280, 'height': 720, 'fps': 30.0, 'frame_count': '45733', 'time_base': 1.736111111111111e-05, 'json': '{"streams": [{"index": 0, "codec_name": "h264", "codec_long_name": "H.264 / AVC / MPEG-4 AVC / MPEG-4 part 10", "profile": "High", "codec_type": "video", "codec_tag_string": "avc1", "codec_tag": "0x31637661", "width": 1280, "height": 720, "coded_width": 1280, "coded_height": 720, "closed_captions": 0, "has_b_frames": 0, "pix_fmt": "yuv420p", "level": 31, "color_range": "tv", "color_space": "bt709", "color_transfer": "bt709", "color_primaries": "bt709", "chroma_location": "left", "refs": 1, "is_avc": "true", "nal_length_size": "4", "r_frame_rate": "30/1", "avg_frame_rate": "30/1", "time_base": "1/57600", "start_pts": 0, "start_time": "0.000000", "duration_ts": 87807360, "duration": "1524.433333", "bit_rate": "443801", "bits_per_raw_sample": "8", "nb_frames": "45733", "disposition": {"default": 1, "dub": 0, "original": 0, "comment": 0, "lyrics": 0, "karaoke": 0, "forced": 0, "

In [23]:
import av
from fractions import Fraction
from typing import Any, Dict, Optional
import subprocess, json

@daft.func(return_dtype = dt.struct({
    "width": dt.int32(),
    "height": dt.int32(),
    "fps": dt.float64(),
    "frame_count": dt.int32(),
    "time_base": dt.float64(),
    "json": dt.string(),
}))
def probe_video_meta(file: daft.File) -> Dict[str, Optional[float | int]]:
    """
    Returns {'width', 'height', 'fps', 'frame_count'} using only container/stream headers.
    Frame count is exact if the stream exposes nb_frames; otherwise estimated via duration * fps.
    """

    out = subprocess.check_output([
        "ffprobe","-v","error",
        "-select_streams","v:0",
        "-show_streams","-show_format",
        "-print_format","json", str("file:///content/videos/Running.mp4")
    ])
    info = json.loads(out)

    # Cast ffprobe output to our dtype
    w = info["streams"][0]["width"]
    h = info["streams"][0]["height"]
    fps = float(Fraction(info["streams"][0]["r_frame_rate"]))
    nb_frames = int(info["streams"][0]["nb_frames"])
    time_base = float(Fraction(info["streams"][0]["time_base"]))

    return {
        "width": w,
        "height": h,
        "fps": fps,
        "frame_count": nb_frames,
        "time_base": time_base,
        "json": json.dumps(info),
        }

In [24]:
df_meta = df_files.with_column("metadata", probe_video_meta(df_files["file"])).collect()
df_meta.show(5)

pathUtf8,sizeInt64,num_rowsInt64,fileFile,"metadataStruct[width: Int32, height: Int32, fps: Float64, frame_count: Int32, time_base: Float64, json: Utf8]"
file:///content/videos/Running.mp4,132936424,None,"File(Reference(""file:///content/videos/Running.mp4"", None))","{width: 1280,height: 720,fps: 30,frame_count: 45733,time_base: 0.00001736111111111111,json: {""streams"": [{""index"": 0, ""codec_name"": ""h264"", ""codec_long_name"": ""H.264 / AVC / MPEG-4 AVC / MPEG-4 part 10"", ""profile"": ""High"", ""codec_type"": ""video"", ""codec_tag_string"": ""avc1"", ""codec_tag"": ""0x31637661"", ""width"": 1280, ""height"": 720, ""coded_width"": 1280, ""coded_height"": 720, ""closed_captions"": 0, ""has_b_frames"": 0, ""pix_fmt"": ""yuv420p"", ""level"": 31, ""color_range"": ""tv"", ""color_space"": ""bt709"", ""color_transfer"": ""bt709"", ""color_primaries"": ""bt709"", ""chroma_location"": ""left"", ""refs"": 1, ""is_avc"": ""true"", ""nal_length_size"": ""4"", ""r_frame_rate"": ""30/1"", ""avg_frame_rate"": ""30/1"", ""time_base"": ""1/57600"", ""start_pts"": 0, ""start_time"": ""0.000000"", ""duration_ts"": 87807360, ""duration"": ""1524.433333"", ""bit_rate"": ""443801"", ""bits_per_raw_sample"": ""8"", ""nb_frames"": ""45733"", ""disposition"": {""default"": 1, ""dub"": 0, ""original"": 0, ""comment"": 0, ""lyrics"": 0, ""karaoke"": 0, ""forced"": 0, ""hearing_impaired"": 0, ""visual_impaired"": 0, ""clean_effects"": 0, ""attached_pic"": 0, ""timed_thumbnails"": 0}, ""tags"": {""creation_time"": ""2025-08-28T01:11:17.000000Z"", ""language"": ""und"", ""handler_name"": ""mp4-muxer-hdlr"", ""vendor_id"": ""[0][0][0][0]""}}], ""format"": {""filename"": ""file:///content/videos/Running.mp4"", ""nb_streams"": 2, ""nb_programs"": 0, ""format_name"": ""mov,mp4,m4a,3gp,3g2,mj2"", ""format_long_name"": ""QuickTime / MOV"", ""start_time"": ""0.000000"", ""duration"": ""1524.458000"", ""size"": ""132936424"", ""bit_rate"": ""697619"", ""probe_score"": 100, ""tags"": {""major_brand"": ""isom"", ""minor_version"": ""512"", ""compatible_brands"": ""isomiso2avc1mp41"", ""creation_time"": ""2025-08-28T01:11:17.000000Z"", ""encoder"": ""Lavf59.16.100""}}},}"


In [26]:
@daft.func(return_dtype=dt.list(dt.float32()))
def clip_frame_time(clip_indices: list[int], time_base: float, fps: float) -> list[int]:
    return [float(i) * time_base for i in clip_indices]


In [27]:
# plan clips
df_clip_plan = (
    df_meta
    .with_column("clip_indices",
        col("metadata")["frame_count"]
        .apply(lambda n: list(range(0, n)), return_dtype=dt.list(dt.int32()))
        .list.chunk(size=T)
    )
    .explode("clip_indices")
    .with_column("clip_time_base", clip_frame_time(col("clip_indices"), col("metadata")["time_base"], col("metadata")["fps"]))
)
df_clip_plan.show(5)

pathUtf8,sizeInt64,num_rowsInt64,fileFile,"metadataStruct[width: Int32, height: Int32, fps: Float64, frame_count: Int32, time_base: Float64, json: Utf8]",clip_indicesFixedSizeList[Int32; 16],clip_time_baseList[Float32]
file:///content/videos/Running.mp4,132936424,None,"File(Reference(""file:///content/videos/Running.mp4"", None))","{width: 1280,height: 720,fps: 30,frame_count: 45733,time_base: 0.00001736111111111111,json: {""streams"": [{""index"": 0, ""codec_name"": ""h264"", ""codec_long_name"": ""H.264 / AVC / MPEG-4 AVC / MPEG-4 part 10"", ""profile"": ""High"", ""codec_type"": ""video"", ""codec_tag_string"": ""avc1"", ""codec_tag"": ""0x31637661"", ""width"": 1280, ""height"": 720, ""coded_width"": 1280, ""coded_height"": 720, ""closed_captions"": 0, ""has_b_frames"": 0, ""pix_fmt"": ""yuv420p"", ""level"": 31, ""color_range"": ""tv"", ""color_space"": ""bt709"", ""color_transfer"": ""bt709"", ""color_primaries"": ""bt709"", ""chroma_location"": ""left"", ""refs"": 1, ""is_avc"": ""true"", ""nal_length_size"": ""4"", ""r_frame_rate"": ""30/1"", ""avg_frame_rate"": ""30/1"", ""time_base"": ""1/57600"", ""start_pts"": 0, ""start_time"": ""0.000000"", ""duration_ts"": 87807360, ""duration"": ""1524.433333"", ""bit_rate"": ""443801"", ""bits_per_raw_sample"": ""8"", ""nb_frames"": ""45733"", ""disposition"": {""default"": 1, ""dub"": 0, ""original"": 0, ""comment"": 0, ""lyrics"": 0, ""karaoke"": 0, ""forced"": 0, ""hearing_impaired"": 0, ""visual_impaired"": 0, ""clean_effects"": 0, ""attached_pic"": 0, ""timed_thumbnails"": 0}, ""tags"": {""creation_time"": ""2025-08-28T01:11:17.000000Z"", ""language"": ""und"", ""handler_name"": ""mp4-muxer-hdlr"", ""vendor_id"": ""[0][0][0][0]""}}], ""format"": {""filename"": ""file:///content/videos/Running.mp4"", ""nb_streams"": 2, ""nb_programs"": 0, ""format_name"": ""mov,mp4,m4a,3gp,3g2,mj2"", ""format_long_name"": ""QuickTime / MOV"", ""start_time"": ""0.000000"", ""duration"": ""1524.458000"", ""size"": ""132936424"", ""bit_rate"": ""697619"", ""probe_score"": 100, ""tags"": {""major_brand"": ""isom"", ""minor_version"": ""512"", ""compatible_brands"": ""isomiso2avc1mp41"", ""creation_time"": ""2025-08-28T01:11:17.000000Z"", ""encoder"": ""Lavf59.16.100""}}},}","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]","[0, 0.000017361112, 0.000034722223, 0.000052083335, 0.000069444446, 0.00008680556, 0.00010416667, 0.00012152778, 0.00013888889, 0.00015625, 0.00017361112, 0.00019097222, 0.00020833334, 0.00022569444, 0.00024305556, 0.00026041668]"
file:///content/videos/Running.mp4,132936424,None,"File(Reference(""file:///content/videos/Running.mp4"", None))","{width: 1280,height: 720,fps: 30,frame_count: 45733,time_base: 0.00001736111111111111,json: {""streams"": [{""index"": 0, ""codec_name"": ""h264"", ""codec_long_name"": ""H.264 / AVC / MPEG-4 AVC / MPEG-4 part 10"", ""profile"": ""High"", ""codec_type"": ""video"", ""codec_tag_string"": ""avc1"", ""codec_tag"": ""0x31637661"", ""width"": 1280, ""height"": 720, ""coded_width"": 1280, ""coded_height"": 720, ""closed_captions"": 0, ""has_b_frames"": 0, ""pix_fmt"": ""yuv420p"", ""level"": 31, ""color_range"": ""tv"", ""color_space"": ""bt709"", ""color_transfer"": ""bt709"", ""color_primaries"": ""bt709"", ""chroma_location"": ""left"", ""refs"": 1, ""is_avc"": ""true"", ""nal_length_size"": ""4"", ""r_frame_rate"": ""30/1"", ""avg_frame_rate"": ""30/1"", ""time_base"": ""1/57600"", ""start_pts"": 0, ""start_time"": ""0.000000"", ""duration_ts"": 87807360, ""duration"": ""1524.433333"", ""bit_rate"": ""443801"", ""bits_per_raw_sample"": ""8"", ""nb_frames"": ""45733"", ""disposition"": {""default"": 1, ""dub"": 0, ""original"": 0, ""comment"": 0, ""lyrics"": 0, ""karaoke"": 0, ""forced"": 0, ""hearing_impaired"": 0, ""visual_impaired"": 0, ""clean_effects"": 0, ""attached_pic"": 0, ""timed_thumbnails"": 0}, ""tags"": {""creation_time"": ""2025-08-28T01:11:17.000000Z"", ""language"": ""und"", ""handler_name"": ""mp4-muxer-hdlr"", "

### Stream Video Frames

In [ ]:
def frame_to_rgb_float32(frame: av.VideoStream, w: int, h: int, interp: str = None) -> np.ndarray:

    frame = frame.to_ndarray(
        width=w,
        height=h,
        format="rgb24",
        interpolation=interp,
    ).astype(np.float32) / 255.0

    return frame

In [ ]:
import matplotlib.pyplot as plt


# Experiment with an example:
my_file = "/content/videos/Why Your Image Processing Pipeline Keeps Running Out of Memory [4CGQ-c7iivg].mkv"


def seek_video_frames(file, start_sec, num_frames, W, H, interp):
    with av.open(file) as container:
        vs = container.streams.video[0]
        vs.thread_type = "AUTO"

        # 1) Compute seek offset in stream ticks
        ts = int(start_sec / float(vs.time_base))  # seconds -> ticks

        # 2) Seek to keyframe <= start_sec
        container.seek(ts, stream=vs, any_frame=False, backward=True)

        # 3) New decode loop; drop until PTS >= start_sec
        out = np.empty((num_frames, H, W, 3), dtype=np.float32)
        got = 0
        target = start_sec
        eps = 1e-6

        for frame in container.decode(video=0):
            if frame.pts is None:
                continue
            t = frame.pts * float(vs.time_base)
            if t + eps < target:
                continue  # not reached start yet
            # 4) Collect frames
            arr = frame_to_rgb_float32(frame, w=W, h=H, interp=interp)
            out[got] = arr
            got += 1
            if got == num_frames:
                break

        # If fewer than requested frames exist, trim
        return out[:got]

stack = seek_video_frames(my_file, 20, 16, 288, 288, None)

# Display the first frame from the stack
for i in range(16):
    plt.subplot(4, 4, i + 1)
    plt.imshow(stack[i])
    plt.axis('off')  # Hide axes

In [ ]:
@daft.func()
def yield_clip_stack(
    file: daft.File,
    start_sec: float,
    num_frames: int = 16,
    w: int = 288,
    h: int = 288,
    interp: str = None
    ) -> dt.tensor(dt.float32(), shape=(T, H, W, 3)):

    with file:
        with av.open(file) as container:
            vs = container.streams.video[0]
            vs.thread_type = "AUTO"

            # 1) Compute seek offset in stream ticks
            ts = int(start_sec / float(vs.time_base))  # seconds -> ticks

            # 2) Seek to keyframe <= start_sec
            container.seek(ts, stream=vs, any_frame=False, backward=True)

            # 3) New decode loop; drop until PTS >= start_sec
            empty_arr = np.empty((num_frames, H, W, 3), dtype=np.float32)
            out = empty_arr.copy()
            got = 0
            target = start_sec
            eps = 1e-6

            for frame in container.decode(video=0):
                if frame.pts is None:
                    continue
                t = frame.pts * float(vs.time_base)
                if t + eps < target:
                    continue  # not reached start yet

                # 4) Collect frames
                arr = frame_to_rgb_float32(frame, w=W, h=H, interp=interp)
                out[got] = arr
                got += 1
                # Check if we have collected enough frames or reached the end of the video
                if got == num_frames or (frame.pts * float(vs.time_base) + eps >= (vs.duration * vs.time_base) if vs.duration and vs.time_base else False):
                    break


            # If fewer than requested frames exist, fill with zeros
            if got < num_frames:
                out[got:] = 0

            return out

In [ ]:
df_clips = (
    df_clip_plan
    .with_column("clip", yield_clip_stack(
            col("file"),
            col("clip_indices").list.get(0),
            num_frames=T,
            w=W,
            h=H
        )
    )
)
df_clips.collect()

### Define Inference Strategy

In [ ]:
B = 8
@daft.udf(
    return_dtype = dt.embedding(dt.float32(), 768),
    batch_size=B, # clips per batch (tune for throughput)
    num_gpus=1,
)
class VideoPrismVideoUDF:
    def __init__(self, model_name: str = "videoprism_lvt_public_v1_base"):
        "for 'videoprism_lvt_public_v1_large', set T = 8"

        from videoprism import models as vp
        self.model = vp.get_model(model_name)
        self.params = vp.load_pretrained_weights(model_name)

        @jax.jit
        def vf_b(clips):  # [B,T,288,288,3] -> [B,D]
            v, _, _ = self.model.apply(
                self.params,
                clips,
                None,
                None,
                train=False
            )
            return v

        self.vf_b = vf_b

        # Warmup both
        _ = self.vf_b(jnp.zeros((B, T, H, W, C), jnp.float32)).block_until_ready()

    def __call__(self,
        clips: list[np.ndarray], # List[T,H,W,C] of len B
    ):
        # Batch Inference
        xb = jnp.stack(clips, axis=0)    # [B,T,H,W,3]
        video_embeddings = self.vf_b(xb) # [B,768]
        np_emb = np.asarray(video_embeddings)  # Back to NumPy
        return [np_emb[i].tolist() for i in range(len(np_emb))]

Recommended GPU Batch sizes:
- T4 (B = 2)
- L4 (B = 2)
- A100 (B = 24)

In [ ]:
import time
start = time.time()
df_video_embs = (
    df_clips
    .with_column("video_embeddings", VideoPrismVideoUDF(col("clip")))
)
df_video_embs.limit(B*2).collect()
print(f"Video Embeddings processed {B} clips, {T} frames in {time.time()-start} sec")


In [ ]:
df_video_embs.select("group_index","video_embeddings", "clip").count_rows()

Extract Audio from Video, Transcribe and Embed

In [ ]:
import av
from av.audio.resampler import AudioResampler

@daft.func()
def extract_audio_frames_into_numpy_arrays(file: daft.File) -> np.ndarray:

    container = av.open(file)
    resampler = AudioResampler(format='s16', layout='mono', rate=16000)

    chunks = []
    try:
        for frame in container.decode(audio=0):
            # Resample to desired SR/mono/PCM16; result can be a frame or list of frames
            res = resampler.resample(frame)
            frames = res if isinstance(res, (list, tuple)) else [res]

            for f in frames:
                arr = f.to_ndarray()  # typically (channels, samples) or (samples,)
                arr = np.asarray(arr)

                # Flatten to 1-D mono
                if arr.ndim == 2:
                    # (1, N) or (N, 1) → (N,)
                    if arr.shape[0] == 1:
                        arr = arr[0]
                    elif arr.shape[1] == 1:
                        arr = arr[:, 0]
                    else:
                        # Unexpected multi-channel after mono resample: average as fallback
                        arr = arr.mean(axis=0)
                elif arr.ndim > 2:
                    arr = arr.reshape(-1)

                # Convert PCM16 → float32 in [-1, 1]
                if arr.dtype != np.float32:
                    arr = (arr.astype(np.float32) / 32768.0).clip(-1.0, 1.0)

                chunks.append(arr)
    finally:
        container.close()

    if not chunks:
        return np.zeros((0,), dtype=np.float32)

    audio = np.concatenate(chunks, axis=0).astype(np.float32, copy=False)
    return audio



In [ ]:
@daft.udf(return_dtype = dt.string())
class ParakeetTranscribeUDF:
    def __init__(self, context_size: int = 256):
        import nemo.collections.asr as nemo_asr
        self.asr_model = nemo_asr.models.ASRModel.from_pretrained(model_name="nvidia/parakeet-tdt-0.6b-v3")
        self.asr_model.change_attention_model(
            self_attention_model="rel_pos_local_attn",
            att_context_size=[context_size, context_size]
        )

    def __call__(self, audio: list[np.ndarray]):
        outputs = self.asr_model.transcribe(audio)
        texts = [o.text for o in outputs]
        return texts



In [ ]:

# Parakeet Transcribe with Timestamps
@daft.udf(return_dtype = dt.struct({
    "segment": dt.list(dt.struct({
        "start_offset": dt.int32(),
        "end_offset": dt.int32(),
        "start": dt.float32(),
        "end": dt.float32()
    })),
}))
class ParakeetTranscribeTimestampsUDF:
    def __init__(self, context_size: int = 256):
        import nemo.collections.asr as nemo_asr
        self.asr_model = nemo_asr.models.ASRModel.from_pretrained(model_name="nvidia/parakeet-tdt-0.6b-v3")
        self.asr_model.change_attention_model(
            self_attention_model="rel_pos_local_attn",
            att_context_size=[context_size, context_size]
        )

    def __call__(self, audio: list[np.ndarray]):
        outputs = self.asr_model.transcribe(audio, timestamps=True)   # No public flag to emit only segments
        return [o.timestamp["segment"] for o in outputs]


In [ ]:


@daft.udf(
    return_dtype = dt.embedding(dt.float32(), 768),
    batch_size=B, # clips per batch (tune for throughput)
    num_gpus=1,
)
class VideoPrismTextUDF:
    def __init__(self, model_name: str = "videoprism_lvt_public_v1_base"):
        from videoprism import models as vp
        self.model = vp.get_model(model_name)
        self.params = vp.load_pretrained_weights(model_name)
        self.text_tokenizer = vp.load_text_tokenizer('c4_en')

        @jax.jit
        def vf_b(text_ids, text_paddings):  # [B,T,288,288,3] -> [B,D]
            _, t, _ = self.model.apply(self.params, None, text_ids, text_paddings, train=False)
            return t # text embeddings

        self.vf_b = vf_b

        # Warmup both
        text_ids, text_paddings = vp.tokenize_texts(self.text_tokenizer, ["Hello", "World"])
        _ = self.vf_b(None, text_ids, text_paddings).block_until_ready()

    def __call__(self,
        prompts: list[str], # List[T,H,W,C] of len B
    ):
        # Batch Inference
        text_ids, text_paddings = vp.tokenize_texts(self.text_tokenizer, prompts)
        text_embeddings = self.vf_b(text_ids, text_paddings)

        return text_embeddings


# Using daft.File instead of read_video_frames()

Video data, as a critical type of multimodal data, uniquely integrates visual, audio, and temporal dimensions, inherently fusing spatial (image-based) and temporal information. It has been widely adopted across domains including short-video platforms, live streaming, public security, healthcare, and autonomous driving.

Given the large volume of video data, most processing paradigms typically involve streaming-based reading and processing to minimize memory footprint. This distinguishes it from image data, which generally requires full initial loading into memory prior to processing.

Thus, when introducing the Video data type into Daft, it should avoid storing the entire dataset in memory. Drawing inspiration from the File data type, we can either store merely a URL reference to the video data or directly utilize the underlying data structure of the File data type as its internal representation.

Beyond the core content of video data, it is critical to extract key metadata to facilitate subsequent filtering of target videos prior to processing. Videos encompass extensive metadata, such as frame count, resolution (height/width), time base, duration, pixel format, bit rate, codec name, and profile, among others. However, incorporating all such metadata into the Video data type is impractical from a memory efficiency standpoint. Instead, we prioritize including only essential metadata fields—specifically frame count, height, width, and FPS. Additional metadata can be dynamically retrieved during video processing as needed.

daft.read_video_frames is a very convenient API for reading video frames, also including recognize/filter the key frames and resize frame, it's very helpful for most common cases, but it might more focus on the case that reading key frames, there are some other cases might need a Video data type and native functions(even though we can used customized UDF to achieve it.), feel free to discuss:
support using different algorithms to reading/extracting accurate key frames, e.g. difference, optical flow, the default behavior of PyAV to extract key frame is based on I-frame the use the native encoding metadata pict_type='I'
Besides extracting/reading key frames, there are other use case about video, e.g. split video by key frame, extract audio from video, etc. it's better to add these functions on Video or file data type instead of adding new API for each use cases.
From performance perspective, it's better to use rust library to handle video processing logical as much as possible, e.g. ffmpeg-next, even though most tools are based on ffmepg
filter/pushdown video data based on thier metadata before processing them.


R Conner Howell
:daft:  Aug 28th at 11:07 AM
I agree. The idea behind the "File" type was to start with a wrapper of the appropriate python file-like protocols, then to further type into VideoFile, AudioFile, PdfFile, etc. — each of which having their own domain-specific methods such a read_frames, read_channels, read_pages etc. respectively. As you have also pointed out, this enables daft to implement this functionality in Rust as well.

In [ ]:
df_file = daft.from_glob_path("~/Movies/")

In [ ]:

df = df.from_glob_path("s3://bucket/videos/")

# Convert path to video directly from utf8 data type to video type. Daft should support convert from utf8 and file data type both.
df = df.with_column("video", video(col("path")))

# Filter video by video metadata.
df = df.filter((df["width"] > 1024) & (df["height"] > 576) & (df["frames"] > 100)))

# Extract the key frames, the `key_frames` function will streaming read video data
# and extract multiple key frames, the data type of each frame is FixedShapeImage. The `key_frames` might add more parameters to indicate what's the image mode of key frames.
# TODO consider whether to include some metadata for key frame to compatible with daft.read_video_frames
df = df.with_column("key_frames", key_frames(col("video"), method= "I_frame").explode("key_frames")

# Save the key frames as a dataset.
df.select("path", "key_frames").write_lance("key_frames_dataset")


## Appendix

In [ ]:
# Parakeet Transcribe with Timestamps
@daft.udf(return_dtype = dt.struct({
    "word": dt.list(dt.struct({
        "word": dt.string(),
        "start_offset": dt.int32(),
        "end_offset": dt.int32(),
        "start": dt.float32(),
        "end": dt.float32()
    })),
    "segment": dt.list(dt.struct({
        "start_offset": dt.int32(),
        "end_offset": dt.int32(),
        "start": dt.float32(),
        "end": dt.float32()
    })),
    "char": dt.list(dt.struct({
        "char": dt.string(),
        "start_offset": dt.int32(),
        "end_offset": dt.int32(),
        "start": dt.float32(),
        "end": dt.float32()
    })),
}))
class ParakeetTranscribeTimestampsUDF:
    def __init__(self, context_size: int = 256):
        import nemo.collections.asr as nemo_asr
        self.asr_model = nemo_asr.models.ASRModel.from_pretrained(model_name="nvidia/parakeet-tdt-0.6b-v3")
        self.asr_model.change_attention_model(
            self_attention_model="rel_pos_local_attn",
            att_context_size=[context_size, context_size]
        )

    def __call__(self, audio: list[np.ndarray]):
        outputs = self.asr_model.transcribe(audio, timestamps=True)   # No public flag to emit only segments
        return [o.timestamp for o in outputs]

In [ ]:
class DiarizationSortFormerUDF:
    def __init__(self, context_size: int = 256):
        from nemo.collections.asr.models import SortformerEncLabelModel
        self.diar_model = SortformerEncLabelModel.from_pretrained("nvidia/diar_streaming_sortformer_4spk-v2")
        self.diar_model.eval() # Switch to inference mode

    def __call__(self, audio: list[np.ndarray]):
        outputs = self.asr_model.transcribe(audio, timestamps=True)   # No public flag to emit only segments
        return [o.timestamp for o in outputs]